In [7]:
import warnings
warnings.filterwarnings(action='once')

import torch
import torch.nn.functional as F
import numpy as np
import json
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import skimage.transform
import argparse
from scipy.misc import imread, imresize
from PIL import Image

/home/pramod/.local/lib/python3.5/site-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.23) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
/home/pramod/anaconda3/envs/py35th/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/pramod/anaconda3/envs/py35th/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
def caption_image_beam_search(encoder, decoder, image_path, word_map, beam_size=3):
    """
    Reads an image and captions it with beam search.

    :param encoder: encoder model
    :param decoder: decoder model
    :param image_path: path to image
    :param word_map: word map
    :param beam_size: number of sequences to consider at each decode-step
    :return: caption, weights for visualization
    """

    k = beam_size
    vocab_size = len(word_map)

    # Read image and process
    img = imread(image_path)
    if len(img.shape) == 2:
        img = img[:, :, np.newaxis]
        img = np.concatenate([img, img, img], axis=2)
    img = imresize(img, (256, 256))
    img = img.transpose(2, 0, 1)
    img = img / 255.
    img = torch.FloatTensor(img).to(device)
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    transform = transforms.Compose([normalize])
    image = transform(img)  # (3, 256, 256)

    # Encode
    image = image.unsqueeze(0)  # (1, 3, 256, 256)
    encoder_out = encoder(image)  # (1, enc_image_size, enc_image_size, encoder_dim)
    enc_image_size = encoder_out.size(1)
    encoder_dim = encoder_out.size(3)

    # Flatten encoding
    encoder_out = encoder_out.view(1, -1, encoder_dim)  # (1, num_pixels, encoder_dim)
    num_pixels = encoder_out.size(1)

    # We'll treat the problem as having a batch size of k
    encoder_out = encoder_out.expand(k, num_pixels, encoder_dim)  # (k, num_pixels, encoder_dim)

    # Tensor to store top k previous words at each step; now they're just <start>
    k_prev_words = torch.LongTensor([[word_map['<start>']]] * k).to(device)  # (k, 1)

    # Tensor to store top k sequences; now they're just <start>
    seqs = k_prev_words  # (k, 1)

    # Tensor to store top k sequences' scores; now they're just 0
    top_k_scores = torch.zeros(k, 1).to(device)  # (k, 1)

    # Tensor to store top k sequences' alphas; now they're just 1s
    seqs_alpha = torch.ones(k, 1, enc_image_size, enc_image_size).to(device)  # (k, 1, enc_image_size, enc_image_size)

    # Lists to store completed sequences, their alphas and scores
    complete_seqs = list()
    complete_seqs_alpha = list()
    complete_seqs_scores = list()

    # Start decoding
    step = 1
    h, c = decoder.init_hidden_state(encoder_out)

    # s is a number less than or equal to k, because sequences are removed from this process once they hit <end>
    while True:

        embeddings = decoder.embedding(k_prev_words).squeeze(1)  # (s, embed_dim)

        awe, alpha = decoder.attention(encoder_out, h)  # (s, encoder_dim), (s, num_pixels)

        alpha = alpha.view(-1, enc_image_size, enc_image_size)  # (s, enc_image_size, enc_image_size)

        gate = decoder.sigmoid(decoder.f_beta(h))  # gating scalar, (s, encoder_dim)
        awe = gate * awe

        h, c = decoder.decode_step(torch.cat([embeddings, awe], dim=1), (h, c))  # (s, decoder_dim)

        scores = decoder.fc(h)  # (s, vocab_size)
        scores = F.log_softmax(scores, dim=1)

        # Add
        scores = top_k_scores.expand_as(scores) + scores  # (s, vocab_size)

        # For the first step, all k points will have the same scores (since same k previous words, h, c)
        if step == 1:
            top_k_scores, top_k_words = scores[0].topk(k, 0, True, True)  # (s)
        else:
            # Unroll and find top scores, and their unrolled indices
            top_k_scores, top_k_words = scores.view(-1).topk(k, 0, True, True)  # (s)

        # Convert unrolled indices to actual indices of scores
        prev_word_inds = top_k_words / vocab_size  # (s)
        next_word_inds = top_k_words % vocab_size  # (s)

        # Add new words to sequences, alphas
        seqs = torch.cat([seqs[prev_word_inds], next_word_inds.unsqueeze(1)], dim=1)  # (s, step+1)
        seqs_alpha = torch.cat([seqs_alpha[prev_word_inds], alpha[prev_word_inds].unsqueeze(1)],
                               dim=1)  # (s, step+1, enc_image_size, enc_image_size)

        # Which sequences are incomplete (didn't reach <end>)?
        incomplete_inds = [ind for ind, next_word in enumerate(next_word_inds) if
                           next_word != word_map['<end>']]
        complete_inds = list(set(range(len(next_word_inds))) - set(incomplete_inds))

        # Set aside complete sequences
        if len(complete_inds) > 0:
            complete_seqs.extend(seqs[complete_inds].tolist())
            complete_seqs_alpha.extend(seqs_alpha[complete_inds].tolist())
            complete_seqs_scores.extend(top_k_scores[complete_inds])
        k -= len(complete_inds)  # reduce beam length accordingly

        # Proceed with incomplete sequences
        if k == 0:
            break
        seqs = seqs[incomplete_inds]
        seqs_alpha = seqs_alpha[incomplete_inds]
        h = h[prev_word_inds[incomplete_inds]]
        c = c[prev_word_inds[incomplete_inds]]
        encoder_out = encoder_out[prev_word_inds[incomplete_inds]]
        top_k_scores = top_k_scores[incomplete_inds].unsqueeze(1)
        k_prev_words = next_word_inds[incomplete_inds].unsqueeze(1)

        # Break if things have been going on too long
        if step > 50:
            break
        step += 1

    i = complete_seqs_scores.index(max(complete_seqs_scores))
    seq = complete_seqs[i]
    alphas = complete_seqs_alpha[i]

    return seq, alphas

In [10]:
#warnings.filterwarnings('ignore')
# Load model
checkpoint = torch.load("models/BEST_checkpoint_coco_5_cap_per_img_5_min_word_freq.pth.tar")
decoder = checkpoint['decoder']
decoder = decoder.to(device)
decoder.eval()
encoder = checkpoint['encoder']
encoder = encoder.to(device)
encoder.eval()
print("Loaded encoder and decoder!")

/home/pramod/anaconda3/envs/py35th/lib/python3.5/site-packages/torch/serialization.py:367: SourceChangeWarning: source code of class 'models.Encoder' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/pramod/anaconda3/envs/py35th/lib/python3.5/site-packages/torch/serialization.py:367: SourceChangeWarning: source code of class 'models.DecoderWithAttention' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/pramod/anaconda3/envs/py35th/lib/python3.5/site-packages/torch/serialization.py:367: SourceChangeWarning: source code of class 'models.Attention' has changed. you can retrieve the original source code by accessing the o

Loaded encoder and decoder!


In [13]:
# Load word map (word2ix)
with open("models/WORDMAP_coco_5_cap_per_img_5_min_word_freq.json", 'r') as j:
    word_map = json.load(j)
rev_word_map = {v: k for k, v in word_map.items()}  # ix2word

In [14]:
seq, alphas = caption_image_beam_search(encoder, decoder, "example.png", word_map, beam_size=5)

/home/pramod/anaconda3/envs/py35th/lib/python3.5/site-packages/ipykernel_launcher.py:17: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/home/pramod/anaconda3/envs/py35th/lib/python3.5/site-packages/scipy/misc/pilutil.py:165: DeprecationWarning: `fromimage` is deprecated!
`fromimage` is deprecated in SciPy 1.0.0. and will be removed in 1.2.0.
Use ``np.asarray(im)`` instead.
  return fromimage(im, flatten=flatten, mode=mode)
/home/pramod/anaconda3/envs/py35th/lib/python3.5/site-packages/ipykernel_launcher.py:21: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/home/pramod/anaconda3/envs/py35th/lib/python3.5/site-packages/scipy/misc/pilutil.py:555: DeprecationWarning: `toimage` is deprecated!
`toimage` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use Pillow's ``Image.

In [4]:
import pandas as pd
import os

In [33]:
show_tell_attend_df  = pd.DataFrame()

images = "./imgs/"
for img in os.listdir(images):
        if os.path.splitext(img)[1] == '.png' or os.path.splitext(img)[1] =='.jpeg' or os.path.splitext(img)[1] =='.jpg':
            print(img)
            seq, _ = caption_image_beam_search(encoder, decoder, "./imgs/"+img, word_map, beam_size=1)
            print(seq)
            words = [rev_word_map[ind] for ind in seq]
            print(words)
            show_tell_attend_df=show_tell_attend_df.append(
            pd.DataFrame({"img":img,"show_attend_tell":str(words)},index=[0]),ignore_index=True)
            

img3.jpg
[9488, 1, 288, 289, 3, 1, 395, 28, 603, 409, 6, 14, 37, 9489]
['<start>', 'a', 'city', 'street', 'with', 'a', 'building', 'and', 'cars', 'parked', 'on', 'the', 'side', '<end>']
img4.jpg
[9488, 1, 123, 3, 1, 102, 28, 333, 28, 1, 1713, 9489]
['<start>', 'a', 'room', 'with', 'a', 'table', 'and', 'chairs', 'and', 'a', 'suitcase', '<end>']
messi.jpg
[9488, 1, 2, 32, 1, 4, 33, 35, 704, 2015, 9489]
['<start>', 'a', 'man', 'in', 'a', 'red', 'shirt', 'is', 'playing', 'frisbee', '<end>']
person.jpg
[9488, 1, 2, 28, 1, 974, 179, 78, 79, 1, 1112, 9489]
['<start>', 'a', 'man', 'and', 'a', 'dog', 'are', 'looking', 'at', 'a', 'sheep', '<end>']
scream.jpg


/home/pramod/anaconda3/envs/py35th/lib/python3.5/site-packages/ipykernel_launcher.py:17: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/home/pramod/anaconda3/envs/py35th/lib/python3.5/site-packages/scipy/misc/pilutil.py:165: DeprecationWarning: `fromimage` is deprecated!
`fromimage` is deprecated in SciPy 1.0.0. and will be removed in 1.2.0.
Use ``np.asarray(im)`` instead.
  return fromimage(im, flatten=flatten, mode=mode)
/home/pramod/anaconda3/envs/py35th/lib/python3.5/site-packages/ipykernel_launcher.py:21: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/home/pramod/anaconda3/envs/py35th/lib/python3.5/site-packages/scipy/misc/pilutil.py:555: DeprecationWarning: `toimage` is deprecated!
`toimage` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use Pillow's ``Image.

[9488, 1, 911, 17, 1, 38, 6, 1, 230, 9489]
['<start>', 'a', 'painting', 'of', 'a', 'woman', 'on', 'a', 'train', '<end>']
dog.jpg
[9488, 1, 974, 35, 98, 6, 1, 3735, 61, 23, 1, 13, 9489]
['<start>', 'a', 'dog', 'is', 'sitting', 'on', 'a', 'leash', 'next', 'to', 'a', 'bike', '<end>']
eagle.jpg
[9488, 1, 416, 3, 1, 91, 1349, 71, 6, 1, 1094, 9489]
['<start>', 'a', 'bird', 'with', 'a', 'long', 'beak', 'standing', 'on', 'a', 'beach', '<end>']
giraffe.jpg
[9488, 1, 1051, 28, 1, 1301, 32, 1, 999, 26, 9489]
['<start>', 'a', 'giraffe', 'and', 'a', 'zebra', 'in', 'a', 'grassy', 'area', '<end>']
herd_of_horses.jpg
[9488, 1, 96, 17, 794, 71, 32, 1, 1324, 798, 716, 9489]
['<start>', 'a', 'group', 'of', 'horses', 'standing', 'in', 'a', 'dry', 'grass', 'field', '<end>']
img1.jpg
[9488, 1, 2, 3, 1, 4070, 28, 1, 4, 710, 9489]
['<start>', 'a', 'man', 'with', 'a', 'beard', 'and', 'a', 'red', 'tie', '<end>']
img2.jpg
[9488, 1, 230, 235, 371, 230, 237, 119, 1, 716, 9489]
['<start>', 'a', 'train', 'traveling'

In [24]:
words = [rev_word_map[ind] for ind in seq]
words

['<start>',
 'a',
 'herd',
 'of',
 'giraffe',
 'standing',
 'on',
 'top',
 'of',
 'a',
 'dirt',
 'road',
 '<end>']

In [34]:
results = pd.read_csv("yolo_sat.csv",names=["img","yolo","show_tell"])
results.head()

,img,yolo,show_tell
NaN,img,yolo,sat
0.0,000000001584.jpg,"['person', 'person', 'person', 'person', 'bus'...",<start> a red and white bus driving down a str...
1.0,giraffe.jpg,"['zebra', 'giraffe', 'giraffe']",<start> a giraffe standing in a field next to ...
2.0,img4.jpg,"['chair', 'chair', 'chair', 'clock']","<start> a living room with a couch , table , a..."
3.0,000000001296.jpg,"['person', 'person', 'cell phone']",<start> a woman is eating a piece of cake on a...


In [35]:
pd.merge(results,show_tell_attend_df,on="img").to_csv("yolo_st_sat_.csv")